In [1]:
import pandas as pd
import numpy as np
import re
import itertools
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import warnings
warnings.filterwarnings("ignore")

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df_main = pd.read_csv(r'N:\spotify_content_based\data\data.csv')
df_genre = pd.read_csv(r'N:\spotify_content_based\data\data_w_genres.csv')

In [4]:
df_main.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


In [5]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170653 entries, 0 to 170652
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   valence           170653 non-null  float64
 1   year              170653 non-null  int64  
 2   acousticness      170653 non-null  float64
 3   artists           170653 non-null  object 
 4   danceability      170653 non-null  float64
 5   duration_ms       170653 non-null  int64  
 6   energy            170653 non-null  float64
 7   explicit          170653 non-null  int64  
 8   id                170653 non-null  object 
 9   instrumentalness  170653 non-null  float64
 10  key               170653 non-null  int64  
 11  liveness          170653 non-null  float64
 12  loudness          170653 non-null  float64
 13  mode              170653 non-null  int64  
 14  name              170653 non-null  object 
 15  popularity        170653 non-null  int64  
 16  release_date      17

In [6]:
df_genre.head()

,genres,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count
0,['show tunes'],"""Cats"" 1981 Original London Cast",0.590111,0.467222,250318.555556,0.394003,0.011400,0.290833,-14.448000,0.210389,117.518111,0.389500,38.333333,5,1,9
1,[],"""Cats"" 1983 Broadway Cast",0.862538,0.441731,287280.000000,0.406808,0.081158,0.315215,-10.690000,0.176212,103.044154,0.268865,30.576923,5,1,26
2,[],"""Fiddler On The Roof” Motion Picture Chorus",0.856571,0.348286,328920.000000,0.286571,0.024593,0.325786,-15.230714,0.118514,77.375857,0.354857,34.857143,0,1,7
3,[],"""Fiddler On The Roof” Motion Picture Orchestra",0.884926,0.425074,262890.962963,0.245770,0.073587,0.275481,-15.639370,0.123200,88.667630,0.372030,34.851852,0,1,27
4,[],"""Joseph And The Amazing Technicolor Dreamcoat""...",0.510714,0.467143,270436.142857,0.488286,0.009400,0.195000,-10.236714,0.098543,122.835857,0.482286,43.000000,5,1,7


In [7]:
df_genre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28680 entries, 0 to 28679
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   genres            28680 non-null  object 
 1   artists           28680 non-null  object 
 2   acousticness      28680 non-null  float64
 3   danceability      28680 non-null  float64
 4   duration_ms       28680 non-null  float64
 5   energy            28680 non-null  float64
 6   instrumentalness  28680 non-null  float64
 7   liveness          28680 non-null  float64
 8   loudness          28680 non-null  float64
 9   speechiness       28680 non-null  float64
 10  tempo             28680 non-null  float64
 11  valence           28680 non-null  float64
 12  popularity        28680 non-null  float64
 13  key               28680 non-null  int64  
 14  mode              28680 non-null  int64  
 15  count             28680 non-null  int64  
dtypes: float64(11), int64(3), object(2)
memo

In [8]:
df_genre['genres_upd'] = df_genre['genres'].apply(lambda x: [re.sub(' ','_',i) for i in re.findall(r"'([^']*)'", x)])

In [9]:
df_genre['genres_upd'].values[100][0]

'new_wave_pop'

In [10]:
type(df_genre['genres_upd'].values[100])

list

In [11]:
df_main['artists_upd_v1'] = df_main['artists'].apply(lambda x: re.findall(r"'([^']*)'", x))

In [12]:
df_main['artists_upd_v1'].values[10000]

['Karen Dalton']

In [13]:
df_main[df_main['artists_upd_v1'].apply(lambda x: not x)].head(5)

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,artists_upd_v1
143,0.300,1921,0.772,"[""Scarlet D'Carpio""]",0.560,249370,0.313,0,7b4eHImKQ51DYaQvNTdtEp,0.000005,6,0.115,-8.346,0,Himno Nacional del Perú,0,1921-09-23,0.0376,107.501,[]
234,0.902,1923,0.994,"[""King Oliver's Creole Jazz Band""]",0.708,194533,0.361,0,1xEEYhWxT4WhDQdxfPCT8D,0.883000,0,0.103,-11.764,0,Snake Rag,20,1923,0.0441,105.695,[]
238,0.554,1923,0.996,"[""King Oliver's Creole Jazz Band""]",0.546,170827,0.189,0,3rauXVLOOM5BlxWqUcDpkg,0.908000,0,0.339,-15.984,1,Chimes Blues,13,1923,0.0581,80.318,[]
244,0.319,1923,0.995,"[""Clarence Williams' Blue Five""]",0.520,197493,0.153,0,1UdqHVRFYMZKU2Q7xkLtYc,0.131000,0,0.353,-14.042,1,Pickin' On Your Baby,11,1923,0.0440,102.937,[]
249,0.753,1923,0.994,"[""King Oliver's Creole Jazz Band""]",0.359,187227,0.357,0,5SvyP1ZeJX1jA7AOZD08NA,0.819000,3,0.290,-11.810,1,Tears,10,1923,0.0511,205.053,[]


In [14]:
df_main['artists_upd_v2'] = df_main['artists'].apply(lambda x: re.findall('\"(.*?)\"',x))
df_main['artists_upd'] = np.where(df_main['artists_upd_v1'].apply(lambda x: not x), df_main['artists_upd_v2'], df_main['artists_upd_v1'] )

In [15]:
df_main['artists_song'] = df_main.apply(lambda row: row['artists_upd'][0]+row['name'],axis = 1)

In [16]:
df_main.sort_values(['artists_song','release_date'], ascending = False, inplace = True)

In [17]:
df_main[df_main['name']=='Yesterday']

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,artists_upd_v1,artists_upd_v2,artists_upd,artists_song
89700,0.0594,2007,0.944000,['Rockabye Baby!'],0.368,175147,0.138,0,6JT9qYS4dCxV0sZgYB5e57,0.892000,5,0.1100,-17.489,1,Yesterday,48,2007-03-13,0.0307,140.604,[Rockabye Baby!],[],[Rockabye Baby!],Rockabye Baby!Yesterday
64259,0.2290,1966,0.924000,['Perry Como'],0.377,182027,0.237,0,5b0xRsYa4LiSJIzyJnDmlO,0.177000,0,0.3850,-15.630,1,Yesterday,33,1966,0.0313,92.546,[Perry Como],[],[Perry Como],Perry ComoYesterday
152745,0.1040,2005,0.198000,['Mary Mary'],0.460,320040,0.538,0,58TvareCLSfzfN8WFWbaWK,0.000002,1,0.0870,-7.100,1,Yesterday,38,2005-11-29,0.0318,139.063,[Mary Mary],[],[Mary Mary],Mary MaryYesterday
152825,0.1580,2006,0.482000,['Kenny G'],0.332,179573,0.420,0,0DPUhwShX0kQQvrG4A0CCD,0.052900,5,0.1480,-7.658,1,Yesterday,50,2006-05-01,0.0324,151.470,[Kenny G],[],[Kenny G],Kenny GYesterday
144896,0.3900,1966,0.752000,['John Denver'],0.497,135840,0.153,0,3fnDgDubGjWY4Dtp99P4zI,0.000000,1,0.0879,-12.153,1,Yesterday,17,1966,0.0270,87.095,[John Denver],[],[John Denver],John DenverYesterday
169727,0.6910,2016,0.701000,['J Dilla'],0.599,71427,0.396,0,0Vg1lQpk2IHk2JU8UfAa5d,0.820000,5,0.1100,-8.519,0,Yesterday,55,2016-06-06,0.1380,104.068,[J Dilla],[],[J Dilla],J DillaYesterday
114614,0.2570,1968,0.098900,['Gladys Knight & The Pips'],0.513,209867,0.193,0,4vRvD2YGAIKV1lxB3i3wMO,0.000002,5,0.2870,-14.927,1,Yesterday,24,1968,0.0339,127.772,[Gladys Knight & The Pips],[],[Gladys Knight & The Pips],Gladys Knight & The PipsYesterday
64909,0.0662,1969,0.581000,['Frank Sinatra'],0.217,239400,0.258,0,3UR7rApi0s3GxDOdOgCKcQ,0.000016,0,0.1110,-12.820,1,Yesterday,37,1969-03,0.0361,129.405,[Frank Sinatra],[],[Frank Sinatra],Frank SinatraYesterday
69943,0.3600,1994,0.736000,['Boyz II Men'],0.338,188267,0.147,0,3x6ElAgBpyvhGcWRQl7Pwh,0.000000,11,0.3740,-15.110,1,Yesterday,40,1994-01-01,0.0299,139.301,[Boyz II Men],[],[Boyz II Men],Boyz II MenYesterday
133177,0.6320,1982,0.000414,['Bad Religion'],0.473,159093,0.954,0,10ulbdcVA6ptYfxbEYpAah,0.000007,0,0.0884,-4.899,0,Yesterday,28,1982,0.0667,120.878,[Bad Religion],[],[Bad Religion],Bad ReligionYesterday


In [18]:
df_main.drop_duplicates('artists_song',inplace = True)

In [19]:
df_main[df_main['name']=='Yesterday']

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,artists_upd_v1,artists_upd_v2,artists_upd,artists_song
89700,0.0594,2007,0.944000,['Rockabye Baby!'],0.368,175147,0.138,0,6JT9qYS4dCxV0sZgYB5e57,0.892000,5,0.1100,-17.489,1,Yesterday,48,2007-03-13,0.0307,140.604,[Rockabye Baby!],[],[Rockabye Baby!],Rockabye Baby!Yesterday
64259,0.2290,1966,0.924000,['Perry Como'],0.377,182027,0.237,0,5b0xRsYa4LiSJIzyJnDmlO,0.177000,0,0.3850,-15.630,1,Yesterday,33,1966,0.0313,92.546,[Perry Como],[],[Perry Como],Perry ComoYesterday
152745,0.1040,2005,0.198000,['Mary Mary'],0.460,320040,0.538,0,58TvareCLSfzfN8WFWbaWK,0.000002,1,0.0870,-7.100,1,Yesterday,38,2005-11-29,0.0318,139.063,[Mary Mary],[],[Mary Mary],Mary MaryYesterday
152825,0.1580,2006,0.482000,['Kenny G'],0.332,179573,0.420,0,0DPUhwShX0kQQvrG4A0CCD,0.052900,5,0.1480,-7.658,1,Yesterday,50,2006-05-01,0.0324,151.470,[Kenny G],[],[Kenny G],Kenny GYesterday
144896,0.3900,1966,0.752000,['John Denver'],0.497,135840,0.153,0,3fnDgDubGjWY4Dtp99P4zI,0.000000,1,0.0879,-12.153,1,Yesterday,17,1966,0.0270,87.095,[John Denver],[],[John Denver],John DenverYesterday
169727,0.6910,2016,0.701000,['J Dilla'],0.599,71427,0.396,0,0Vg1lQpk2IHk2JU8UfAa5d,0.820000,5,0.1100,-8.519,0,Yesterday,55,2016-06-06,0.1380,104.068,[J Dilla],[],[J Dilla],J DillaYesterday
114614,0.2570,1968,0.098900,['Gladys Knight & The Pips'],0.513,209867,0.193,0,4vRvD2YGAIKV1lxB3i3wMO,0.000002,5,0.2870,-14.927,1,Yesterday,24,1968,0.0339,127.772,[Gladys Knight & The Pips],[],[Gladys Knight & The Pips],Gladys Knight & The PipsYesterday
64909,0.0662,1969,0.581000,['Frank Sinatra'],0.217,239400,0.258,0,3UR7rApi0s3GxDOdOgCKcQ,0.000016,0,0.1110,-12.820,1,Yesterday,37,1969-03,0.0361,129.405,[Frank Sinatra],[],[Frank Sinatra],Frank SinatraYesterday
69943,0.3600,1994,0.736000,['Boyz II Men'],0.338,188267,0.147,0,3x6ElAgBpyvhGcWRQl7Pwh,0.000000,11,0.3740,-15.110,1,Yesterday,40,1994-01-01,0.0299,139.301,[Boyz II Men],[],[Boyz II Men],Boyz II MenYesterday
133177,0.6320,1982,0.000414,['Bad Religion'],0.473,159093,0.954,0,10ulbdcVA6ptYfxbEYpAah,0.000007,0,0.0884,-4.899,0,Yesterday,28,1982,0.0667,120.878,[Bad Religion],[],[Bad Religion],Bad ReligionYesterday


In [20]:
artists_exploded = df_main[['artists_upd','id']].explode('artists_upd')

In [21]:
artists_exploded_enriched = artists_exploded.merge(df_genre, how = 'left', left_on = 'artists_upd',right_on = 'artists')
artists_exploded_enriched_nonnull = artists_exploded_enriched[~artists_exploded_enriched.genres_upd.isnull()]

In [22]:
artists_exploded_enriched_nonnull[artists_exploded_enriched_nonnull['id'] =='6KuQTIu1KoTTkLXKrwlLPV']

,artists_upd,id,genres,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count,genres_upd


In [23]:
artists_exploded_enriched_nonnull

,artists_upd,id,genres,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count,genres_upd
0,조정현,2ghebdwe2pNXT4eL34T7pW,['classic korean pop'],조정현,0.568000,0.447000,237688.000000,0.215000,0.000001,0.064900,-16.478000,0.027200,71.979000,0.177000,31.000000,10.0,1.0,2.0,[classic_korean_pop]
1,黑豹,3KIuCzckjdeeVuswPo20mC,"['chinese indie', 'chinese indie rock']",黑豹,0.381000,0.353000,316160.000000,0.686000,0.000000,0.056800,-9.103000,0.039500,200.341000,0.352000,35.000000,11.0,1.0,2.0,"[chinese_indie, chinese_indie_rock]"
2,黃國隆,4prhqrLXYMjHJ6vpRAlasx,[],黃國隆,0.785455,0.570818,174582.727273,0.148400,0.000083,0.142191,-21.610091,0.054355,119.586273,0.741273,23.000000,5.0,1.0,11.0,[]
3,黃國隆,5xFXTvnEe03SyvFpo6pEaE,[],黃國隆,0.785455,0.570818,174582.727273,0.148400,0.000083,0.142191,-21.610091,0.054355,119.586273,0.741273,23.000000,5.0,1.0,11.0,[]
4,王秋玉,5xFXTvnEe03SyvFpo6pEaE,[],王秋玉,0.885000,0.649500,150620.000000,0.115000,0.000000,0.092750,-21.182500,0.066100,121.090000,0.696500,23.000000,7.0,0.0,4.0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206532,"""Weird Al"" Yankovic",2Ec0liSMY2h4XORHyzKCmY,"['comedy rock', 'comic', 'parody']","""Weird Al"" Yankovic",0.173145,0.662787,218948.196721,0.695393,0.000050,0.161102,-9.768705,0.084536,133.031180,0.751344,34.229508,9.0,1.0,122.0,"[comedy_rock, comic, parody]"
206533,"""Weird Al"" Yankovic",2zMUYAHJfDU5MKqoCusTwo,"['comedy rock', 'comic', 'parody']","""Weird Al"" Yankovic",0.173145,0.662787,218948.196721,0.695393,0.000050,0.161102,-9.768705,0.084536,133.031180,0.751344,34.229508,9.0,1.0,122.0,"[comedy_rock, comic, parody]"
206534,"""Weird Al"" Yankovic",1Yd0HypJ0bzilJS9MBztZZ,"['comedy rock', 'comic', 'parody']","""Weird Al"" Yankovic",0.173145,0.662787,218948.196721,0.695393,0.000050,0.161102,-9.768705,0.084536,133.031180,0.751344,34.229508,9.0,1.0,122.0,"[comedy_rock, comic, parody]"
206535,"""Weird Al"" Yankovic",2CstSu0GDI4dUFtE3VdubW,"['comedy rock', 'comic', 'parody']","""Weird Al"" Yankovic",0.173145,0.662787,218948.196721,0.695393,0.000050,0.161102,-9.768705,0.084536,133.031180,0.751344,34.229508,9.0,1.0,122.0,"[comedy_rock, comic, parody]"


In [24]:
artists_genres_consolidated = artists_exploded_enriched_nonnull.groupby('id')['genres_upd'].apply(list).reset_index()

In [25]:
artists_genres_consolidated['consolidates_genre_lists'] = artists_genres_consolidated['genres_upd'].apply(lambda x: list(set(list(itertools.chain.from_iterable(x)))))

In [26]:
artists_genres_consolidated.head()

,id,genres_upd,consolidates_genre_lists
0,000G1xMMuwxNHmwVsBdtj1,"[[candy_pop, dance_rock, new_wave, new_wave_po...","[new_wave, dance_rock, new_wave_pop, power_pop..."
1,000GyYHG4uWmlXieKLij8u,"[[alternative_hip_hop, conscious_hip_hop, minn...","[conscious_hip_hop, minnesota_hip_hop, alterna..."
2,000Npgk5e2SgwGaIsN3ztv,"[[classic_bollywood, classic_pakistani_pop, fi...","[sufi, classic_pakistani_pop, filmi, ghazal, c..."
3,000ZxLGm7jDlWCHtcXSeBe,"[[boogie-woogie, piano_blues, ragtime, stride]]","[ragtime, boogie-woogie, piano_blues, stride]"
4,000jBcNljWTnyjB4YO7ojf,[[]],[]


In [27]:
df_main = df_main.merge(artists_genres_consolidated[['id','consolidates_genre_lists']], on = 'id',how = 'left')

In [28]:
df_main.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,consolidates_genre_lists
0,0.177,1989,0.568,['조정현'],0.447,237688,0.2150,0,2ghebdwe2pNXT4eL34T7pW,0.000001,10,0.0649,-16.478,1,그아픔까지사랑한거야,31,1989-06-15,0.0272,71.979,[조정현],[],[조정현],조정현그아픔까지사랑한거야,[classic_korean_pop]
1,0.352,1992,0.381,['黑豹'],0.353,316160,0.6860,0,3KIuCzckjdeeVuswPo20mC,0.000000,11,0.0568,-9.103,1,DON'T BREAK MY HEART,35,1992-12-22,0.0395,200.341,[黑豹],[],[黑豹],黑豹DON'T BREAK MY HEART,"[chinese_indie, chinese_indie_rock]"
2,0.458,1963,0.987,['黃國隆'],0.241,193480,0.0437,0,4prhqrLXYMjHJ6vpRAlasx,0.000453,5,0.2650,-24.571,1,藝旦調,23,1963-05-28,0.0443,85.936,[黃國隆],[],[黃國隆],黃國隆藝旦調,[]
3,0.796,1963,0.852,"['黃國隆', '王秋玉']",0.711,145720,0.1110,0,5xFXTvnEe03SyvFpo6pEaE,0.000000,2,0.0695,-20.741,0,草螟弄雞公,23,1963-05-28,0.0697,124.273,"[黃國隆, 王秋玉]",[],"[黃國隆, 王秋玉]",黃國隆草螟弄雞公,[]
4,0.704,1963,0.771,['黃國隆'],0.610,208760,0.1750,0,6Pqs2suXEqCGx7Lxg5dlrB,0.000000,7,0.0309,-20.232,1,思想起,23,1963-05-28,0.0419,124.662,[黃國隆],[],[黃國隆],黃國隆思想起,[]


In [29]:
df_main.tail()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,consolidates_genre_lists
156602,0.768,1997,0.2820,"[""Lil' Kim"", ""Lil' Cease""]",0.748,275947,0.693,0,2LP2uDQQ7eLMcUVE4aOpAV,0.00000,11,0.123,-7.832,0,Crush on You (feat. Lil' Cease) - Remix,56,1997-06-30,0.2780,88.802,"[ Kim"", ""Lil]","[Lil' Kim, Lil' Cease]","[ Kim"", ""Lil]","Kim"", ""LilCrush on You (feat. Lil' Cease) - R...",NaN
156603,0.792,2004,0.0248,"[""Lil' Flip"", 'Lea']",0.814,225173,0.387,1,4s0o8TJHfX9LLHa0umnOzT,0.00000,0,0.131,-9.867,1,Sunshine (feat. Lea),62,2004-03-30,0.0945,93.961,"[ Flip"", ]",[Lil' Flip],"[ Flip"", ]","Flip"", Sunshine (feat. Lea)",NaN
156604,0.697,1999,0.0516,"[""Ol' Dirty Bastard"", 'Kelis', 'Rich Travali']",0.934,239547,0.459,1,6YYd5MLpu45J0uLrMdivF7,0.00000,1,0.222,-7.654,1,Got Your Money (feat. Kelis),66,1999,0.1890,103.040,"[ Dirty Bastard"", , , ]",[Ol' Dirty Bastard],"[ Dirty Bastard"", , , ]","Dirty Bastard"", Got Your Money (feat. Kelis)",NaN
156605,0.429,1994,0.0249,"[""World Class Wreckin' Cru"", ""Michel 'Le""]",0.715,351040,0.490,0,3hoiinUc5VA9xUEJID7R8V,0.00017,9,0.139,-9.504,0,Turn Off The Lights - Rap,36,1994-04-06,0.0479,129.309,"[ Cru"", ""Michel ]","[World Class Wreckin' Cru, Michel 'Le]","[ Cru"", ""Michel ]","Cru"", ""Michel Turn Off The Lights - Rap",NaN
156606,0.273,1996,0.0113,"[""Rappin' 4-Tay"", 'MC Breed', 'Too $hort']",0.897,337973,0.414,1,78859Af0fmA9VTlgnOHTAP,0.00011,4,0.101,-8.450,0,Never Talk Down,35,1996,0.2460,96.039,"[ 4-Tay"", , , ]",[Rappin' 4-Tay],"[ 4-Tay"", , , ]","4-Tay"", Never Talk Down",NaN


In [30]:
df_main['year'] = df_main['release_date'].apply(lambda x: x.split('-')[0])

In [31]:
float_cols = df_main.dtypes[df_main.dtypes == 'float64'].index.values

In [32]:
ohe_cols = 'popularity'

In [33]:
df_main['popularity'].describe()

count    156607.000000
mean         31.307215
std          21.712234
min           0.000000
25%          11.000000
50%          33.000000
75%          48.000000
max         100.000000
Name: popularity, dtype: float64

In [34]:
df_main['popularity_red'] = df_main['popularity'].apply(lambda x: int(x/5))

In [35]:
df_main['consolidates_genre_lists'] = df_main['consolidates_genre_lists'].apply(lambda d: d if isinstance(d, list) else [])

In [36]:
df_main.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,consolidates_genre_lists,popularity_red
0,0.177,1989,0.568,['조정현'],0.447,237688,0.2150,0,2ghebdwe2pNXT4eL34T7pW,0.000001,10,0.0649,-16.478,1,그아픔까지사랑한거야,31,1989-06-15,0.0272,71.979,[조정현],[],[조정현],조정현그아픔까지사랑한거야,[classic_korean_pop],6
1,0.352,1992,0.381,['黑豹'],0.353,316160,0.6860,0,3KIuCzckjdeeVuswPo20mC,0.000000,11,0.0568,-9.103,1,DON'T BREAK MY HEART,35,1992-12-22,0.0395,200.341,[黑豹],[],[黑豹],黑豹DON'T BREAK MY HEART,"[chinese_indie, chinese_indie_rock]",7
2,0.458,1963,0.987,['黃國隆'],0.241,193480,0.0437,0,4prhqrLXYMjHJ6vpRAlasx,0.000453,5,0.2650,-24.571,1,藝旦調,23,1963-05-28,0.0443,85.936,[黃國隆],[],[黃國隆],黃國隆藝旦調,[],4
3,0.796,1963,0.852,"['黃國隆', '王秋玉']",0.711,145720,0.1110,0,5xFXTvnEe03SyvFpo6pEaE,0.000000,2,0.0695,-20.741,0,草螟弄雞公,23,1963-05-28,0.0697,124.273,"[黃國隆, 王秋玉]",[],"[黃國隆, 王秋玉]",黃國隆草螟弄雞公,[],4
4,0.704,1963,0.771,['黃國隆'],0.610,208760,0.1750,0,6Pqs2suXEqCGx7Lxg5dlrB,0.000000,7,0.0309,-20.232,1,思想起,23,1963-05-28,0.0419,124.662,[黃國隆],[],[黃國隆],黃國隆思想起,[],4


In [37]:
df_main.to_csv(r'N:\spotify_content_based\data\data_updated.csv')

In [38]:
def ohe_prep(df, column, new_name): 
    
    tf_df = pd.get_dummies(df[column])
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)    
    return tf_df


In [39]:
def create_feature_set(df, float_cols):
    
    #tfidf genre lists
    tfidf = TfidfVectorizer()
    tfidf_matrix =  tfidf.fit_transform(df['consolidates_genre_lists'].apply(lambda x: " ".join(x)))
    genre_df = pd.DataFrame(tfidf_matrix.toarray())
    genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names()]
    genre_df.reset_index(drop = True, inplace=True)

    # one hot encoding    
    year_ohe = ohe_prep(df, 'year','year') * 0.5 # 0.5 before

    #scaling float columns
    floats = df[float_cols].reset_index(drop = True)
    floats['mode'] = df['mode']
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.3 # 0.2 0.3 before

    #concanenate all features
    final = pd.concat([genre_df, floats_scaled, year_ohe], axis = 1)
     
    #add song id
    final['id']=df['id'].values
    
    return final

In [40]:
complete_feature_set= create_feature_set(df_main, float_cols=float_cols)

In [41]:
complete_feature_set.tail()

genre|21st_century_classical  genre|432hz  genre|_hip_hop  \
156602                           0.0          0.0             0.0   
156603                           0.0          0.0             0.0   
156604                           0.0          0.0             0.0   
156605                           0.0          0.0             0.0   
156606                           0.0          0.0             0.0   

        genre|a_cappella  genre|abstract  genre|abstract_beats  \
156602               0.0             0.0                   0.0   
156603               0.0             0.0                   0.0   
156604               0.0             0.0                   0.0   
156605               0.0             0.0                   0.0   
156606               0.0             0.0                   0.0   

        genre|abstract_hip_hop  genre|accordeon  genre|accordion  \
156602                     0.0              0.0              0.0   
156603                     0.0              0.0              0.0   
156604                     0.0              0.0              0.0   
156605                     0.0              0.0              0.0   
156606                     0.0              0.0              0.0   

        genre|acid_house  genre|acid_rock  genre|acid_trance  \
156602               0.0              0.0                0.0   
156603               0.0              0.0                0.0   
156604               0.0              0.0                0.0   
156605               0.0              0.0                0.0   
156606               0.0              0.0                0.0   

        genre|acousmatic  genre|acoustic  genre|acoustic_blues  \
156602               0.0             0.0                   0.0   
156603               0.0             0.0                   0.0   
156604               0.0             0.0                   0.0   
156605               0.0             0.0                   0.0   
156606               0.0             0.0                   0.0   

        genre|acoustic_pop  genre|acoustic_punk  genre|acoustic_rock  \
156602                 0.0                  0.0                  0.0   
156603                 0.0                  0.0                  0.0   
156604                 0.0                  0.0                  0.0   
156605                 0.0                  0.0                  0.0   
156606                 0.0                  0.0                  0.0   

        genre|action_rock  genre|adoracion  genre|adult_standards  \
156602                0.0              0.0                    0.0   
156603                0.0              0.0                    0.0   
156604                0.0              0.0                    0.0   
156605                0.0              0.0                    0.0   
156606                0.0              0.0                    0.0   

        genre|adventista  genre|afghan_pop  genre|afghan_traditional  \
156602               0.0               0.0                       0.0   
156603               0.0               0.0                       0.0   
156604               0.0               0.0                       0.0   
156605               0.0               0.0                       0.0   
156606               0.0               0.0                       0.0   

        genre|african  genre|african_percussion  genre|african_reggae  \
156602            0.0                       0.0                   0.0   
156603            0.0                       0.0                   0.0   
156604            0.0                       0.0                   0.0   
156605            0.0                       0.0                   0.0   
156606            0.0                       0.0                   0.0   

        genre|african_rock  genre|afro  genre|afro_dancehall  \
156602                 0.0         0.0                   0.0   
156603                 0.0         0.0                   0.0   
156604                 0.0         0.0                   0.0   
156605                 0.0   

In [42]:
complete_feature_set.to_csv(r'N:\spotify_content_based\data\data_feature_set.csv')

In [43]:
complete_feature_set.shape

(156607, 3051)

In [44]:
complete_feature_set['genre|alternative_rock'].mean()

0.010371076864169637

In [45]:
complete_feature_set.shape

(156607, 3051)

In [46]:
cols = pd.Series(complete_feature_set.columns)

In [47]:
genres = cols[cols.apply(lambda x: bool(re.search('genre', x)))].apply(lambda x:re.split('genre\|', x)[1])

In [48]:
genres.reset_index(drop=True, inplace=True)

In [49]:
genres

0       21st_century_classical
1                        432hz
2                     _hip_hop
3                   a_cappella
4                     abstract
                 ...          
2935                      zolo
2936                   zouglou
2937                      zouk
2938              zurich_indie
2939                    zydeco
Length: 2940, dtype: object

In [50]:
years = cols[cols.apply(lambda x: bool(re.search('year', x)))].apply(lambda x:re.split('year\|', x)[1])

In [51]:
years.reset_index(drop=True, inplace=True)

In [52]:
years.max()

'2020'